# MATH 300: Numerical Analysis I Recitation

## Instructor: Liam Doherty

## Meeting Time/Place: F 11:00AM, Curtis 453

## Office Hours: MRC Hours MR 5p-7p, W 12p-2p, or in Korman 282 by appointment

We'll start today with answering any questions about Project 1, or anything about class material and assignments.

After questions, we'll have some fun! We'll look at a simplified version of an applied problem that I have encountered in my research that uses a kind of interpolation, so that we can see how concepts we are covering in class are used in a "real-world" scenario. In the process, we'll learn basic operation of some important Julia packages, including `Plots.jl` and `DifferentialEquations.jl`.

Now to the fun! Consider the following differential equation model of a decaying material (e.g., radioactive decay): $\newline \newline$
$$
\frac{d}{dt}u(t) = -ku(t), \;\; u(0) = 1.
\newline
$$
From your ODE class, recall that this differential equation has the solution $u(t) = e^{-kt}$. The two parameters in this model are the initial amount of substance $u(0)$ and the rate of decay $k$. What we'd like to find out is what the amount of substance is at time $t = 1$, i.e., to compute $u(1)$.

Now, an interesting problem: What if we don't know the decay rate $k$? This is, of course, possible in practice, because we may be modeling an unknown material (in which case we don't know its properties), or our substance may be impure (and the impurities may cause some variance in the decay rate of the overall compound). Then we won't be able to directly solve this problem! However, there is a way that we can still produce some meaningful results for the problem. The trick is interpolation!

What we'll do is solve the ODE for **several** values of $k$ which approximate reality (some values we use may be less than the "real" $k$, some greater) and then interpolate between the solutions we get as a function of $k$ (so we'll get a dataset $\{(k_{0}, u(1; k_{0})), (k_{1}, u(1; k_{1})), \dots, (k_{n}, u(1; k_{n}))\}$ and then interpolate this data).

Let's start by seeing how to (numerically) solve this ODE using `DifferentialEquations.jl` for one value of $k$:

In [ ]:
using DifferentialEquations
using Plots

# Pick a particular k and put it in a parameter vector p
p = [1.]

# Define right hand side of du/dt = f(u, p, t) (p contains problem parameters)
f(u, p, t) = -p[1]*u

# Define initial condition
u0 = 1.

# Define time to integrate (start at 0, and since we want solution at t = 1, finish at t = 1)
tspan = (0., 1.)

# Wrap everything into an ODEProblem data structure (provided by DifferentialEquations.jl)
prob = ODEProblem(f, u0, tspan, p)

# Solve the problem
sol = solve(prob, reltol = 1e-15)

# Plot solution (plot recipes overriden for ODE solution objects)
display(plot(sol, title = "Solution of Decay ODE", ylabel = "u(t)", label = "u(t)"))

# Extract the amount of substance at the final time t = 1
final_amount = sol.u[end]
println("Amount of substance at time 1: $(final_amount)")

Now, this is the solution for a _particular_ $k$, but we wanted to solve it for many different values of $k$. We could do this with a `for` loop over a vector of different $k$ and plot all the solutions, but `DifferentialEquations.jl` offers an interface for doing [Ensemble Simulations](https://diffeq.sciml.ai/stable/features/ensemble/#ensemble) that makes the coding a bit cleaner (and allows for easy parallelization, which makes it much faster for large numbers of ODEs!). We can set that up in the following way:

In [ ]:
# Define k values to solve for
k_vals = LinRange(0, 1, 11)

# Function to remake the problem with a new k
function prob_func(prob, i, repeat)
    @. prob.p = [k_vals[i]]
    return prob
end

# Put all the ODE problems in one structure
ensemble_prob = EnsembleProblem(prob, prob_func = prob_func)

# Solve all the problems at the same time
sim = solve(ensemble_prob, trajectories = 11, reltol = 1e-15)

# Plot all the solutions at the same time
display(plot(sim, title = "Solutions for Different Parameters", ylabel = "u(t; k)",
    label = ["k = $(k_vals[i])" for i in (1:length(k_vals))'], legend = :outertopright))

# Find out what all of the end solutions are for each k
final_states = [(k_vals[i], sim[i].u[end]) for i in 1:length(k_vals)]

We can also show these points as a scatter plot against $k$:

In [ ]:
scatter(final_states, xlabel = "k", ylabel = "u(1; k)",
        title = "Final State by k Value", legend = :none)

Now, we are set up in exactly the position that we want to be in to interpolate the data! Remember, for interpolation we need to be given data in $(x, y)$ pairs, which is exactly what we have! We can use `DataInterpolations.jl` to construct Lagrange interpolants given a collection of data.

In [ ]:
using DataInterpolations

# Degree of the polynomial is one less than the number of data points
n = length(final_states) - 1

# Get final u values and their correspoding k values in their own vectors
us = [final_states[i][2] for i in 1:length(final_states)]
ks = [final_states[i][1] for i in 1:length(final_states)]

# Construct the Lagrange interpolant
interpolant = LagrangeInterpolation(us, ks, n)

# Evaluate the interpolant on a fine grid and plot
fine_grid = LinRange(0, 1, 100)
plot!(fine_grid, interpolant.(fine_grid))

Now, we can use this interpolant to make guesses about what the solution of the ODE would be with a decay rate that we haven't used! For example, let's look at what happens when we predict the final solution at $k = 0.55$:

In [ ]:
estimated_solution = interpolant(0.55)

Now, compare to the actual solution:

In [ ]:
# Pick a particular k and put it in a parameter vector p
p = [0.55]

# Define right hand side of du/dt = f(u, p, t) (p contains problem parameters)
f(u, p, t) = -p[1]*u

# Define initial condition
u0 = 1.

# Define time to integrate (start at 0, and since we want solution at t = 1, finish at t = 1)
tspan = (0., 1.)

# Wrap everything into an ODEProblem data structure (provided by DifferentialEquations.jl)
prob = ODEProblem(f, u0, tspan, p)

# Solve the problem
sol = solve(prob, reltol = 1e-15)

# Extract the final state at time 1
final_state = sol.u[end]

What's our absolute error?

In [ ]:
abs(estimated_solution - final_state)

Pretty good! Also, recalling our solution as a function of $k$ is given by $u(1; k) = e^{-k}$, we can use the Lagrange error bound from Theorem 3.3. This would give (replacing $u(1; x)$ by $f(x)$ for notational simplicity) $\newline \newline$
$$
\begin{align*}
    |f(x) - P(x)| &\leq \max_{x \in [0, 1]}\Big|\frac{f^{(11)}(\xi(x))}{11!}\prod_{i = 0}^{10}(x - x_{i})\Big| \\
    &\leq \frac{1}{11!}\max_{x \in [0, 1]}\big|x(x- 0.1)(x - 0.2)\dots(x - 0.9)(x - 1)\big| \\
    &\leq \frac{1}{11!} (4.16614 \times 10^{-6}) \approx 1.044 \times 10^{-13},
\end{align*}
\newline
$$
which captures our error.

We could get even better results than this if we invested more up front to solve the ODE at more interpolation nodes (this method is sometimes called **(stochastic) collocation**). Furthermore, notice that we have reduced the computational cost for running the problem for getting a solution with a new $k$; now, instead of solving an ODE (which can be very expensive with large or stiff systems!), all we have to do is evaluate a polynomial.

In [ ]:
using BenchmarkTools

println("Timing interpolant...")
@btime interpolant(0.55)

println("Timing ODE solve...")
@btime solve(prob);

Notice that timing the interpolant, we get just under 1 microsecond, whereas the ODE solve takes about 5 times as long (and runs with almost 10 times the allocations!). This is another benefit of using interpolation; if we can reasonably simplify our problem to evaluating some interpolant, it can lead to **dramatic** performance improvements for future evaluations with a little bit of computational cost upfront to construct the interpolant. Generally, this technique of using one function in place of another function is called using a **surrogate**.